In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[1]:

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In[2]:

make function to check url rquest and parse url content ant retern soup

In [ ]:
def get_url(url):
    #request url 
    url1=requests.get(str(url))
    
    #parse url content
    soup=BeautifulSoup(url1.content,"html.parser")
    
    #retyrn soup
    return soup 
    

get url request 

In [ ]:
website_soup=get_url("https://www.semsarmasr.com")
#website_soup

In[3]:

et function to extract number from the string using regex

In [ ]:
import re
def return_num(price):
  text=price.replace(",","")
  pattern="[0-9]+"
  price=re.findall(pattern,text)
  return price

In[4]:

In [ ]:
def Opensooq_scraping():
    
    df=pd.DataFrame(columns=["link","bathrooms","bedrooms","city","location1","location2","price","size","price_per_unit","type","unit","lat","lon","view","description","web_name","source","rent_sale"])
    category_list=[7585,7891]
    for category in category_list:
        if category==7585:
            page_num=1
        else:
            page_num=1
      while (True):
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
        url = (f"https://eg.opensooq.com/ar/find?page={page_num}&PostSearch[categoryId]={category}")
        print(url)
        result = requests.get(url, headers=headers)
        src = result.content
        soup = BeautifulSoup(src, "lxml")

        #find apartment or every advertsment container------------------------------------------------------------------------------
        apartments=soup.find("div",{"class","pt-16"}).find_all("div",{"class":"mb-32 relative"})

        #loop in every adverts to extract data----------------------------------------------------------------------------------------
        links=[]
        for apartment in apartments:
          #find link of adverts -------------------------------------------------------------link-------------------------------------
          link="https://eg.opensooq.com"+apartment.find("a").attrs["href"]
          print(f"the link of adverts: {link}")
          links.append(link)
          adverts_soup=get_url(link)
          #find rent_sale----------------------------------------------------------------rent_sale------------------------------------------------------
          if category==7585:
            rent_sale="للبيع"
            print(f"the rent_sale : {rent_sale}")
          else:
            rent_sale="للإيجار"
            print(f"the rent_sale : {rent_sale}")

In [ ]:
          #find unit----------------------------------------------------------------unit------------------------------------------------------
          unit=apartment.find("div",{"class":"sc-265a2526-3 zoIKj postDet flex-1 flex flexSpaceBetween flexDirectionColumn"}).find("div",{"class":"flex alignItems"}).find("div",{"class":"me-8 category"}).text.split("في")[0]
          print(f"the unit of apart : {unit}")
          #find descryption-----------------------------------------------------------descryption----------------------------------------------------------
          descryptions=apartment.find("div",{"class":"sc-265a2526-3 zoIKj postDet flex-1 flex flexSpaceBetween flexDirectionColumn"}).find("div",{"class":"flex flexSpaceBetween"}).find("h2").text
          print(f"the descryption is : {descryptions}")
          #find table which have data in adverts---------------------------------------------------------------table------------------------
          try:
            table=adverts_soup.find("div",{"class":"sc-23286f3d-0 cluGaS mt-32 mb-32 border radius-8 p-16"}).find_all("div",{"class":"flex flexSpaceBetween flexWrap mt-8"})
          except:
            pass
          #intialize values of data ------------------------------------------------------------------------------------------------------------------------
          bathrooms=None
          bedrooms=None
          city=None
          location1=None
          price=None
          size=None
          price_per_unit=None

In [ ]:
          #find price ------------------------------------------------------------price-------------------------------------------------------------
          try:
            price=adverts_soup.find("section",{"class":"sc-a8a57ece-2 cvAeHJ sideBar pb-16"}).find("div",{"class":"sc-fab7f819-0 caVLzG radius-8"}).find("span").text
            price=return_num(price)[0]
            print(f"the price is: {price}")
          except:
            price=None
            print(f"the price is :{price}")
          #loop in table of content -------------------------------------------------------------------------------------------------------------------
          for data in table:
            #find bathrooms--------------------------------------------------------------------------------------bathrooms-------------------------------
            try:
              if data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[1].find("p").text=="عدد الحمامات":
                bathrooms=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[1].find("a").text.replace("حمّامين","2").replace("حمّامات","").replace("حمّام","1").replace("+","")
                print(f"the num_bathroom is: {bathrooms}")
            except:
              pass

In [ ]:
            #find bedrooms------------------------------------------------------------------------------------------------bedrooms-----------------------------------------------------------------
            try:
              if data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[0].find("p").text=="عدد الغرف":
                bedrooms=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[0].find("a").text.replace("غرفة نوم","1").replace("غرف نوم","").replace("غرفتا نوم","").replace("غرف","").replace("+","")
                #bedrooms=return_num(bedrooms)
                print(f"the num_bedrooms is: {bedrooms}")
            except:
              pass

            #find city------------------------------------city------------------------------------------------------------------------
            try:
              if data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[0].find("p").text=="المدينة":
                city=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[0].find("a").text
                #bedrooms=return_num(bedrooms)
                print(f"the city is: {city}")
            except:
              pass

            #find size------------------------------------size-------------------------------------------------------------
            try:
              if data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[1].find("p").text=="مساحة الأرض":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[1].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
              elif data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[0].find("p").text=="مساحة الأرض":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[0].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
              elif data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[1].find("p").text=="مساحة البناء":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[1].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
              elif data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[0].find("p").text=="مساحة البناء":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[0].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
            except:
              pass

            #find size--------------------------------------------------------------------size-------------------------------------------------------------------
            try:
              if data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[1].find("p").text=="مساحة البناء":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[1].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
              elif data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[1].find("p").text=="مساحة الأرض":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[1].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
              elif data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[0].find("p").text=="مساحة البناء":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[0].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
              elif data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[0].find("p").text=="مساحة الأرض":
                size=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems"})[0].find("a").text
                size=return_num(size)[0]
                print(f"the size is: {size}")
            except:
              pass

            #find location1------------------------------------location1----------------------------------------------------------------------------------------
            try:
              if data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[1].find("p").text=="الحي / المنطقة":
                location1=data.find_all("div",{"class":"sc-23286f3d-1 hhPCva flex flexSpaceBetween alignItems lightGrayBg"})[1].find("a").text
                #bedrooms=return_num(bedrooms)
                print(f"the location1 is: {location1}")
            except:
              pass
          #find price_per_unit------------------------------------------------------------------price_per_unite---------------------------------------------
          try:
            price_per_unit=int(price)/int(size)
            print(f"the price_per_unit is : {price_per_unit}")
          except:
            price_per_unit=None
            print(f"the price_per_unit is : {price_per_unit}")      

In [ ]:
            #find bathrooms---------------------------------------------------------------------------------------bathrooms---------------------------
          df=df.append({"link":link,
                          "bathrooms":bathrooms,
                        "bedrooms":bedrooms,
                        "city":city,
                        "location1":location1,
                        "location2":location1,
                        "price":price,
                        "size":size,
                        "price_per_unit":price_per_unit,
                        "unit":unit,
                        "web_name":"OpenSooq",
                        "source":"scraping",
                        "description":descryptions,
                        "rent_sale":rent_sale
                        },ignore_index=True)
          df.to_csv("sooqelmftoh_N2.csv",encoding="utf-8-sig",index=False)

In [ ]:
        next_page=soup.find("div",{"id":"pagination"}).find("a",{"class":"sc-c272c79-4 gbyEnK grayBg blueHoverBg pagerItem flex alignItems justifyContent radius-8"})
        print(f"the next page is: {next_page}")
        if next_page !=None:
          print("i am finding the next page !!")
          page_num+=1
        else:
          print(" ohhh idont find the next page!!!!")
          break
            
            

In[5]:

In [ ]:
def Opensooq_clean():
    df=pd.read_csv("sooqelmftoh_N2.csv")
    #clean type--------------------------------------------------
    df.loc[df.unit =="شقق للبيع ",'type'] = "سكني"
    df.loc[df.unit =='فلل - قصور للبيع ', 'type'] = "سكني"
    df.loc[df.unit =='عمارات للبيع ','type'] = "سكني"
    df.loc[df.unit =="بيوت - منازل للبيع ",'type'] = "سكني"
    df.loc[df.unit =="عقارات أجنبية للبيع ", 'type'] = "سكني"
    df.loc[df.unit=="شقق للايجار ","type"]="سكني"
    df.loc[df.unit=="فلل - قصور للايجار ","type"]="سكني"
    df.loc[df.unit=="بيوت - منازل للإيجار ","type"]="سكني"
    df.loc[df.unit=="عمارات للايجار ","type"]="سكني"
    df.loc[df.unit=="غرف ومشاركة سكن ","type"]="سكني"
    df.loc[df.unit=="أبنية تجارية للبيع ","type"]="تجاري"
    df.loc[df.unit=="أبنية تجارية للبيع ","type"]="تجاري"
    df.loc[df.unit=="مزارع وشاليهات للبيع ","type"]="تجاري"
    df.loc[df.unit=="أبنية تجارية للايجار","type"]="تجاري"
    df.loc[df.unit=="تجاري للبيع ","type"]="تجاري"
    df.loc[df.unit=="شقق وأجنحة فندقية ","type"]="سكني"
    df.loc[df.unit=="أبنية تجارية للايجار ","type"]="تجاري"
    df.loc[df.unit=="أراضي للبيع ","type"]="تجاري"
    df.loc[df.unit=="مزارع وشاليهات للإيجار ","type"]="تجاري"
    df.loc[df.unit=="أراضي للإيجار ","type"]="تجاري"

In [ ]:
    #clean unit------------------------------------------
    df.loc[df.unit =="شقق للبيع ",'unit'] = "شقة"
    df.loc[df.unit =='فلل - قصور للبيع ', 'unit'] = "فيلا"
    df.loc[df.unit =='عمارات للبيع ','unit'] = "عمارة"
    df.loc[df.unit =="بيوت - منازل للبيع ",'unit'] = "منزل"
    df.loc[df.unit =="عقارات أجنبية للبيع ", 'unit'] = "عقار"
    df.loc[df.unit=="شقق للايجار ","unit"]="شقة"
    df.loc[df.unit=="فلل - قصور للايجار ","unit"]="فيلا"
    df.loc[df.unit=="بيوت - منازل للإيجار ","unit"]="منزل"
    df.loc[df.unit=="عمارات للايجار ","unit"]="عمارة"
    df.loc[df.unit=="غرف ومشاركة سكن ","unit"]="غرفة"
    df.loc[df.unit=="أبنية تجارية للبيع ","unit"]="أبنية تجارية"
    df.loc[df.unit=="مزارع وشاليهات للبيع ","unit"]="مزارع"
    df.loc[df.unit=="أبنية تجارية للايجار","unit"]="أبنية تجارية"
    df.loc[df.unit=="تجاري للبيع ","unit"]="أبنية تجارية"
    df.loc[df.unit=="شقق وأجنحة فندقية ","unit"]="شقة"
    df.loc[df.unit=="أبنية تجارية للايجار ","unit"]="أبنية تجارية"
    df.loc[df.unit=="أراضي للبيع ","unit"]="ارض"
    df.loc[df.unit=="مزارع وشاليهات للإيجار ","unit"]="مزرعة"
    df.loc[df.unit=="أراضي للإيجار ","unit"]="ارض"
    
    #cleant rent_sale--------------------------------------
    df.loc[df.rent_sale=="للبيع","rent_sale"]="بيع"
    df.loc[df.rent_sale=="للإيجار","rent_sale"]="ايجار"
    df.loc[df.rent_sale=="مطلوب للشراء","rent_sale"]="بيع"
    df.loc[df.rent_sale=="إيجار قانون قديم","rent_sale"]="ايجار"
    
    
    df["description"]=df["description"].str.replace('\n', '')
    df["description"]=df["description"].str.strip()
    
    
    df.loc[df.location1=="أخرى","location1"]=df["city"]
    df.loc[df.location2=="أخرى","location2"]=df["city"]
    
    
    #clean numbathrooms
    #clean bathrooms
    df.loc[df.bathrooms=="٢","bathrooms"]="2"
    df.loc[df.bathrooms=='٤ ',"bathrooms"]="4"
    df.loc[df.bathrooms=='٣ ',"bathrooms"]="3"
    df.loc[df.bathrooms=='٥ ',"bathrooms"]="5"

    #clean bedrooms
    df.loc[df.bedrooms=='٦ ',"bedrooms"]="6"
    df.loc[df.bedrooms=='٤ ',"bedrooms"]="4"
    df.loc[df.bedrooms=='٣ ',"bedrooms"]="3"
    df.loc[df.bedrooms=='٢ ',"bedrooms"]="2"
    df.loc[df.bedrooms=='٥ ',"bedrooms"]="5"
    df.loc[df.bedrooms=='ستوديو',"bedrooms"]=None
    
    df.loc[df.unit=="مزارع","unit"]="مزرعة"

In [ ]:
    #find lat lon --------------------------------------------------------
    from geopy.geocoders import Nominatim

In [ ]:
    geolocator = Nominatim(user_agent="myApp")
    for i in df.index:
        print(i)
        try:
            #tries fetch address from geopy
            location = geolocator.geocode(df["location1"][i]+"-"+df['city'][i]+"-"+"مصر")
            print(location)

            #append lat/long to column using dataframe location
            df['lat'][i] = location.latitude
            df['lon'][i]= location.longitude
        #catches exception for the case where no value is returned
        #appends null value to column
        except:
            df['lat'][i] = ""
            df['lon'][i] = ""

In [ ]:
    print(df["lat"])
    
    sooq_after=df.to_csv("Opeensooq_Clean2.csv",encoding="utf-8-sig",index=False)
    
    return sooq_after

In[6]:

In [ ]:
def Semsar_scraping():
    
    category_list=[765,844,766,898,767,0,770,768,876,769,771]
    df=pd.DataFrame(columns=["link","bathrooms","bedrooms","city","location1","location2","price","size","price_per_unit","type","unit","lat","lon","view","description","web_name","source","rent_sale"])
    links=[]
    for category in category_list:
      print(f"the category is: {category}")
      if category==765:
          page_num=1
      else:
        page_num=1
      while (True):
        print(f"the num_page: {page_num}")
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
        url = (f"https://www.semsarmasr.com/3akarat?title=%CF%E6%C8%E1%DF%D3%20%E1%E1%C8%ED%DA%20%DD%ED%20%E3%D5%D1&r=70&g=0&a=0&cid={category}&p={page_num}")
        print(url)
        result = requests.get(url, headers=headers)
        src = result.content
        soup = BeautifulSoup(src, "lxml")
        try:
          links_in_page = soup.find("div",{"id":"table"}).find_all("div",{"id":"row"})
        except:
          pass
          #print(links_in_page)

        #looop in each page and find links of each adevertsment in page and parse it and extract data from it 
        for apart in links_in_page:

In [ ]:
          l = apart.find("a").attrs['href'] #find link of each advertsment
          print(f"the link of the adves :{l}")
          adverts_soup=get_url(l)  #get soup of each link 
          #find table of content in each advertsment-------------------find tabel--------------------------
          try:
            table= adverts_soup.find("div",{"class":"Row"}).find("div",{"class":"AdDetContainer ListDesStyle AdDetailsStyle"}).find("div",{"class":"table"}).find_all("div",{"id":"DetRrow"})
          except:
            pass
          # intialize the values of this features -----------------------------------------------------------------
          price=None
          area=None  
          num_room=None
          num_bathroom=None
          rent_sale=None
          unit=None
          location1=None
          city=None
          price_per_unite=None
          # loop in table to extract data---------------------table of data-----------------------------------------------------------
          for data in table:
            #find price------------------------price--------------------------------------------------------------------------------
            if data.find_all("div")[0].text=="السعر: ":
              price=data.find_all("div")[1].find("strong").text
              price=return_num(price)
              try:
                price=price[0]
                print("the price is: "+price)
              except:
                price=None
                print(f"the price is :{price}")
            #find size---------------------------size-----------------------------------------------------------------------------------
            elif data.find_all("div")[0].text=="المساحة: ":
              area=data.find_all("div")[1].text
              area=return_num(area)
              area=area[0]
              print("the area is : "+area)

            #find numbers of bedroom----------------num_bedrooms-------------------------------------------------------------------------
            elif data.find_all("div")[0].text=="عدد الغرف: ":
              num_room=data.find_all("div")[1].text
              print("the num_room is: "+num_room)

            #find number of bathrooms----------------num_bathroom-------------------------------------------------------------------------
            elif data.find_all("div")[0].text=="عدد الحمامات: ":
              num_bathroom=data.find_all("div")[1].text
              print("the number of batheroom is: "+num_bathroom)

            #find type of category------------------rent_sale-------------------------------------------------------------------------------
            elif data.find_all("div")[0].text=="الغرض: ":
              rent_sale=data.find_all("div")[1].text
              print("the rent_sale : "+rent_sale)
            #find unit--------------------------------------------------------------------unit-----------------------------------------------
            elif data.find_all("div")[0].text=="القسم: ":
              unit=data.find_all("div")[1].find("a").find("span").text.split(" ")[0]
              print("the unite of category is: "+unit)
          #find price per unite-------------------------------------------------------------------------------------price_per_unite-------------------------------------------
          try:
            price_per_unite=int(float(price)/float(area))
            print(f"the price per unite is :{int(price_per_unite)}")
          except:
            price_per_unite=None
          #find location1------------------------------------------------------location1----------------------------------------
          try:
            city=apart.find_all("div",{"class":"Intcell"})[1].find("div",{"class":"AdLocation"}).text.split("-")[1]
            print("the city is : "+city)
          except:
            pass
          #find location2---------------------------------------------------------------------locatuion2-------------------------------------
          location2=apart.find_all("div",{"class":"Intcell"})[1].find("div",{"class":"AdLocation"}).text
          print("the location2 is: "+location2)
          #find city------------------------------------------------------city---------------------------------------------------
          try:
            location1=apart.find_all("div",{"class":"Intcell"})[1].find("div",{"class":"AdLocation"}).text.split("-")[0]
            print("the ;ocation1 is: "+location1)
          except:
           pass
          #find descryption--------------------------descryption-----------------------------------------------------   
          try:
            description=adverts_soup.find("div",{"class":"PageTitlesStyle"}).find("a").text.replace("إعلان مُميز","") #find descryption 
            print("the description is: "+description)
          except:
            pass
          print("------------------------------------------------>")
          #append data on data frame------------------------------------------------------------------------------------------------
          df=df.append({"link":l,
                         "bathrooms":num_bathroom,
                         "bedrooms":num_room,
                         "price":price,
                         "size":area,
                         "location1":location1,
                         "location2":location1,
                         "city":city,
                         "unit":unit,
                         "rent_sale":rent_sale,
                         "description":description,
                         "web_name":"semsar",
                         "source":"scraping",
                        "price_per_unit":price_per_unite
                         },ignore_index=True)
                        #convert data frame into csv file-------------------------------------------------------------------------
          out=df.to_csv("Semsar3.csv",encoding="utf-8-sig",index=False)
          links.append(l)
        print(len(links))

        #find next page to break while loop if the condition is not true ----------------------------------------------
        try:
          next_page_link = soup.find("div",{"id":"paging"}).find("a",{"class":"notactive"})
          print(next_page_link)
        except:
          next_page_link = soup.find("div",{"id":"paging"}).find("a",{"class":"notactive"})
          print(next_page_link) 

In [ ]:
        if next_page_link==None or next_page_link.text =="❮❮ الأول" :
            print("donesssssssssssss")
            page_num += 1
        else:
            print("-----------End the category is  done---------")
            break
    return out
            

In[7]:

In [ ]:
def Semsar_cleaning():
    
    df=pd.read_csv("Semsar3.csv")

In [ ]:
    #find lat and lond of location
    from geopy.geocoders import Nominatim

In [ ]:
    geolocator = Nominatim(user_agent="myApp")
    for i in df.index:
        print(i)
        try:
            #tries fetch address from geopy
            location = geolocator.geocode(df["location1"][i]+"-"+df['city'][i]+"-"+"مصر")
            print(location)

            #append lat/long to column using dataframe location
            df['lat'][i] = location.latitude
            df['lon'][i]= location.longitude
        #catches exception for the case where no value is returned
        #appends null value to column
        except:
            df['lat'][i] = ""
            df['lon'][i] = ""

In [ ]:
    print(df["lat"])

In [ ]:
    #cleant rent_sale--------------------------------------
    df.loc[df.rent_sale=="للبيع","rent_sale"]="بيع"
    df.loc[df.rent_sale=="للإيجار","rent_sale"]="ايجار"
    df.loc[df.rent_sale=="مطلوب للشراء","rent_sale"]="بيع"
    df.loc[df.rent_sale=="إيجار قانون قديم","rent_sale"]="ايجار"
    
    
    #clean type-----------------------------------
    df.loc[df.unit =="شقق",'type'] = "سكني"
    df.loc[df.unit =="فلل", 'type'] = "سكني"
    df.loc[df.unit =="دوبلكس",'type'] = "سكني"
    df.loc[df.unit =="شاليهات",'type'] = "سكنى"
    df.loc[df.unit =="فنادق", 'type'] = "سكني"
    df.loc[df.unit =="مبانى", 'type'] = "سكني"
    df.loc[df.unit=="بنايات","type"]="سكني"
    df.loc[df.unit=="عمارات","type"]="سكني"
    df.loc[df.unit=="غرف","type"]="سكني"
    df.loc[df.unit=="غرفة","type"]="سكني"
    df.loc[df.unit=="مكاتب","type"]="اداري"
    df.loc[df.unit=="محلات","type"]="تجاري"
    df.loc[df.unit=="جراج","type"]="تجاري"
    df.loc[df.unit=="صيدليات","type"]="تجاري"
    df.loc[df.unit=="عيادات","type"]="تجاري"
    df.loc[df.unit=="قاعات","type"]="تجاري"
    df.loc[df.unit=="مخازن","type"]="تجاري"
    df.loc[df.unit=="مراكز","type"]="تجاري"
    df.loc[df.unit=="مزارع","type"]="تجاري"
    df.loc[df.unit=="مستشفيات","type"]="تجاري"
    df.loc[df.unit=="مغسلة","type"]="تجاري"
    df.loc[df.unit=="مطاعم","type"]="تجاري"
    df.loc[df.unit=="مصانع","type"]="تجاري"
    df.loc[df.unit=="محطة","type"]="تجاري"
    df.loc[df.unit=="مدارس","type"]="تعليمى"
    df.loc[df.unit=="حضانات","type"]="تعليمى"
    df.loc[df.unit=="أراضي","type"]="زراعى"
    df.loc[df.unit=="حدائق","type"]="تجاري"
    df.loc[df.unit=="ستوديو","type"]="تجاري"
    df.loc[df.unit=="مباني","type"]="تجاري"
    
    
    #clean unit -------------------------------------
    df.loc[df.unit =="شقق",'unit'] = "شقة"
    df.loc[df.unit =='فلل', 'unit'] = "فيلا"
    df.loc[df.unit =='عمارات','unit'] = "عمارة"
    df.loc[df.unit =="بيوت - منازل للبيع ",'unit'] = "منزل"
    df.loc[df.unit =="عقارات", 'unit'] = "عقار"
    df.loc[df.unit=="مزارع","unit"]="مزرعة"
    df.loc[df.unit=="عمارات","unit"]="عمارة"
    df.loc[df.unit=="أراضي","unit"]="ارض"
    df.loc[df.unit=="دوبلكس","unit"]="دوبلكس"
    df.loc[df.unit=="شاليهات","unit"]="شاليه"
    df.loc[df.unit=="مكاتب","unit"]="مكتب"
    df.loc[df.unit=="محلات","unit"]="محل"
    df.loc[df.unit=="صيدليات","unit"]="صيدلية"
    df.loc[df.unit=="حضانات","unit"]="حضانة"
    df.loc[df.unit=="مطاعم","unit"]="مطعم و كافيه"
    df.loc[df.unit=="فنادق","unit"]="فندق"
    df.loc[df.unit=="ستوديو","unit"]="ستوديو"
    df.loc[df.unit=="مصانع","unit"]="مصنع"
    df.loc[df.unit=="عيادات","unit"]="عيادة"
    df.loc[df.unit=="مستشفيات","unit"]="مستشفى"
    df.loc[df.unit=="مخازن","unit"]="مخزن"
    df.loc[df.unit=="مراكز","unit"]="مركز"
    df.loc[df.unit=="غرف","unit"]="غرفة"
    df.loc[df.unit=="مباني","unit"]="مبنى"
    df.loc[df.unit=="قاعات","unit"]="قاعة"
    df.loc[df.unit=="مدارس","unit"]="مدرسة"
    df.loc[df.unit=="حدائق","unit"]="حديقة"
    
    
    df.to_csv("Semsar_clean.csv",encoding="utf-8-sig",index=False)
    
    
    

In[8]:

In [ ]:
def tageer_scraping():
    lst=[]                  #list of data
        #list of categories
    category_list=['flats-for-rent','shkk-k-non-kdym','furnished-flats','rooms-for-rent','villas-for-rent','business-rent-4','commercial-properties-4-rent','car-rental']
        #list of all links
    links=[]
    #categories loop
    for category in category_list:
        #counter of pages
        page_num=1   
        while (True) :
            #the headers of the web 
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
            #link of all pages of the website
            url=f'https://tageer.net/{category}?page={page_num}'
            print(url)
            #connecting to the web
            result = requests.get(url, headers=headers)
            src = result.content
            soup = BeautifulSoup(src, "lxml")
            #scraping main tag
            links_in_page = soup.find_all('div',{'class':'caption pull-left'})
            #scraping the link for each main tag
            for i in links_in_page:
                l = i.find("a").attrs['href']                        
                links.append(l)                         #adding link to list
            next_page_link = soup.find("li", {'class':"disabled"})                  #scraping the condition
            if next_page_link != None:            
                print('done')
                break
            else:
                page_num+=1
    #scraping each link
    for link in links:
        d={}
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
        result1 = requests.get(link, headers=headers)
        src1= result1.content
        soup1= BeautifulSoup(src1, "lxml")
        d['link']=link
        print(link)
        d['web_name']='tageer'
        print('tageer')
        try:
            d['price']=soup1.find('div',{'class':'col-xs-12 col-sm-2 price price-curry'}).text.replace('EGP','').strip()
            print(soup1.find('div',{'class':'col-xs-12 col-sm-2 price price-curry'}).text.replace('EGP','').strip())
        except:
            d['price']=None
        try:
            d['unit']=soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[3].text.replace('|','').strip()
            print(soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[3].text.replace('|','').strip())
        except:
            d['unit']=None
        try:
            d['location1']=soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[1].text.replace('|','').strip()
            print(soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[1].text.replace('|','').strip())
        except:
            d['location1']=None
        try:
            d['location2']=soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[2].text.replace('|','').partition('،')[0].strip()
            print(soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[2].text.replace('|','').partition('،')[0].strip())
        except:
            d['location2']=None
        try:
            d['city']=soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[2].text.replace('|','').partition('،')[2].strip()
            print(soup1.find('ul',{'class':'breadcrumb'}).find_all('a')[2].text.replace('|','').partition('،')[2].strip())
        except:
            d['city']=None
        try:
            d['description']=soup1.find('div',{'class':'tab-pane fade active in'}).find('p').text.strip()
            print(soup1.find('div',{'class':'tab-pane fade active in'}).find('p').text.strip())
        except:
            d['description']=None    
        d['source']='scrape'
        print('scrape')

In [ ]:
        try:
            table2=soup1.find('table',{'class':'table table-striped'})        #scraping the table for each link
            table=table2.find_all('tr')
            for i in table:
                try:
                    if i.find_all('td')[0].text=='المساحة بالمتر':
                        d['size']=i.find_all('td')[1].text
                        print(i.find_all('td')[1].text)
                except:
                    d['size']=None
                    print(None)
            for x in table:
                try:
                    if x.find_all('td')[0].text=='عدد الغرف':
                        d['bedrooms']=x.find_all('td')[1].text
                        print(x.find_all('td')[1].text)
                except:
                    d['bedrooms']=None
                    print(None)
            for y in table:
                try:
                    if y.find_all('td')[0].text=='عدد الحمامات':
                        d['bathrooms']=y.find_all('td')[1].text
                        print(y.find_all('td')[1].text)
                except:
                    d['bathrooms']=None
                    print(None)
        except:
            pass
        #adding dictionary to list
        lst.append(d)
        df=pd.DataFrame(lst)#exporting data to csv file
        df.to_csv(r'Tageer2.csv',encoding ='utf-8-sig',index=False)
            

In[9]:

In [ ]:
def tageer_cleaning():
    df=pd.read_csv(r'Tageer2.csv',index_col=False)
    df['all-location'] = df['location2'] + "," +  df['city']+','+'مصر'
    df['lat'] = ""
    df['lon'] = ""
    geolocator = Nominatim(user_agent="myApp")
    for i in df.index:
        try:
            #tries fetch address from geopy
            location = geolocator.geocode(df['all-location'][i])

            #append lat/long to column using dataframe location
            df.loc[i,'lat'] = location.latitude
            df.loc[i,'lon'] = location.longitude
        except:
            #catches exception for the case where no value is returned
            #appends null value to column
            df.loc[i,'lat'] = ""
            df.loc[i,'lon'] = ""
    
    del df['all-location']
    df['rent_sale']='ايجار'
    df['price_per_unit']=''
    df['type']=''
    df['view']=''
    df=df.dropna(subset=['unit'])
    df=df.dropna(subset=['city'])
    df=df.dropna(subset=['price'])
    df=df.dropna(subset=['size'])
    df=df.dropna(subset=['lon'])
    #converting data type
    df['price']=df['price'].str.replace(',','')
    df['price']=df['price'].astype('float')
    df['size']=df['size'].astype('float')
    #replceing with correct values
    df.bedrooms=df.bedrooms.replace({'٥':'5','٤':'4','٧':'7','٦':'6','٩':'9','٨':'8','٢':'2','١':'1','١٠':'10','٣':'3','١٠+':'10','10+':'10'})
#converting data type
    df.bedrooms=df.bedrooms.astype('float')
    df.bathrooms=df.bathrooms.replace({'٥':'5','٤':'4','٧':'7','٦':'6','٩':'9','٨':'8','٢':'2','١':'1','١٠':'10','٣':'3','2.60':'0','10.50':'0','10+':'10','٣+':'3','١٠+':'10'})
    df['bathrooms']=df['bathrooms'].astype('float')
    df=df[['link','bathrooms','bedrooms','city','location1','location2','price','size','price_per_unit','type','unit','lat','lon','view','description','web_name','source','rent_sale']]
    df['price_per_unit']=df['price']/df['size']
    df.unit=df.unit.replace({'شقق للايجار':'شقة','شقق ايجار 59سنة':'شقة','شقق مفروشة للايجار':'شقة','غرف للايجار':'غرفة','فيلات و شاليهات':'فيلا','مشروعات للايجار':'مشروع','عقارات تجارية':'عقار','محلات للايجار':'محل','مكاتب و مبانى للايجار':'مكتب','مخازن واراضي للايجار':'ارض'})
    df.loc[df.unit == 'عقار', 'type'] = "تجاري"
    df.loc[df.unit == 'محل', 'type'] = "تجاري"
    df.loc[df.unit == 'مكتب', 'type'] = "اداري"
    df.loc[df.unit == 'ارض', 'type'] = "تجاري"
    df.loc[df.unit == 'فيلا', 'type'] = "سكني"
    df.loc[df.unit == 'شقة', 'type'] = "سكني"
    df.loc[df.unit == 'غرفة', 'type'] = "سكني"
    df.loc[df.unit == 'مشروع', 'type'] = "تجاري"
    df.drop(df[df['unit']=='سيارات للايجار'].index,inplace=True)
    df.source='scraping'
    return df.to_csv(r'tageer2_cleaning.csv',encoding ='utf-8-sig',index=False)

In[10]:

In [ ]:
def olx_scraping():
    lst=[]                  #list of data
#list of all links
    links=[]
    num_page=1
    while True:
        #header for the web site
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
        #url for the website
        url=f'https://www.olx.com.eg/properties/?page={num_page}'
        print(url)
        #connecting to web site
        result = requests.get(url, headers=headers)
        #content of the web site
        src = result.content
        #html code for the web site
        soup = BeautifulSoup(src, "lxml")
        #scraping links in each link
        links_in_page=soup.find_all('div',{'class':'_41d2b9f3'})
        #print(links_in_page)
        #scraping links for each url
        for i in links_in_page:
            l='https://www.olx.com.eg'+i.find('a').attrs['href']
            links.append(l)
        #condition which counter will stop
        next_page=soup.find('div',{'title':'التالي'}) 
        if next_page!= None:
            num_page+=1
        else:
            print('done')
            break
    #scraping data from each link
    for link in links:
        d={}
        d['link']=link
        print(link)
        d['web_name']='olx'
        d['source']='scraping'
        response=requests.get(link,headers=headers)
        text=response.content
        soup2=BeautifulSoup(text,'html.parser')
        try:
            d['rent_sale']=soup2.find_all('a',{'class':'_151bd34b'})[2].text.partition(' ')[-1]
            print(soup2.find_all('a',{'class':'_151bd34b'})[2].text.partition(' ')[-1])
        except:
            d['rent_sale']=None
            print(d['rent_sale'])
        try:
            d['description']=soup2.find('div',{'class':'_0f86855a'}).text.replace('\n','').strip()
            print(soup2.find('div',{'class':'_0f86855a'}).text)
        except:
            d['description']=None
            print(d['description'])
        try:        
            d['city']=soup2.find('span',{'class':'_8918c0a8'}).text.partition(', ')[2].strip()
            print(d['city'])
        except:
            d['city']=None
            print(d['city'])
        try:        
            d['location2']=soup2.find('span',{'class':'_8918c0a8'}).text.partition(', ')[0].strip().partition('-')[0].strip()
            print(d['location2'])
        except:
            d['location2']=None
            print(d['location2'])
        try:
            location1=soup2.find('span',{'class':'_8918c0a8'}).text.partition(',')[0].partition('-')[2].strip()
            if location1=='':
                d['location1']=d['location2']
                print(d['location1'])
            else:
                d['location1']=location1
                print(d['location1'])
        except:
            d['location1']=None
            print(d['location1'])
        try:
            table2=soup2.find('div',{'class':'_241b3b1e'})        #scraping the table for each link
            table=table2.find_all('div',{'class':"b44ca0b3"})
            d['bedrooms']=None
            d['bathrooms']=None
            for n in table:
                 try:
                    if n.find_all('span')[0].text=='الحمامات':
                        d['bathrooms']=n.find_all('span')[1].text
                        print(n.find_all('span')[1].text)
                 except:
                    d['bathrooms']=None
                    print(None)
                 try:
                    if n.find_all('span')[0].text=='غرف نوم':
                        d['bedrooms']=n.find_all('span')[1].text
                        print(n.find_all('span')[1].text)
                 except:
                    d['bedrooms']=None
                    print(None)
                 try:
                    if n.find_all('span')[0].text=='النوع':
                        d['unit']=n.find_all('span')[1].text
                        print(n.find_all('span')[1].text)
                 except:
                    d['unit']=None
                    print(None)
                 try:
                    if n.find_all('span')[0].text=='المساحة (م٢)':
                        d['size']=n.find_all('span')[1].text
                        print(n.find_all('span')[1].text)
                 except:
                    d['size']=None
                    print(None)
                 try:
                    if n.find_all('span')[0].text=='السعر':
                        d['price']=n.find_all('span')[1].text
                        print(n.find_all('span')[1].text)
                 except:
                    d['price']=None
                    print(None)

In [ ]:
        except:
            pass
        lst.append(d)
        df=pd.DataFrame(lst)#exporting data to csv file
        df.to_csv(r'olx_scraping.csv',encoding ='utf-8-sig',index=False)

In[11]:

In [ ]:
def olx_cleaning():
    df=pd.read_csv(r'olx_scraping.csv',index_col=False)
    df['all-location'] = df['location2'] + "," +  df['city']+','+'مصر'
    df['lat'] = ""
    df['lon'] = ""
    geolocator = Nominatim(user_agent="myApp")
    for i in df.index:
        try:
            #tries fetch address from geopy
            location = geolocator.geocode(df['all-location'][i])

            #append lat/long to column using dataframe location
            df.loc[i,'lat'] = location.latitude
            df.loc[i,'lon'] = location.longitude
        except:
            #catches exception for the case where no value is returned
            #appends null value to column
            df.loc[i,'lat'] = ""
            df.loc[i,'lon'] = ""
    del df['all-location']
    #creating some columns
    df['view']=''
    df['type']=''
    df['price_per_unit']=''
    #replacing values
    df.bedrooms=df.bedrooms.replace('10+','10')
    #converting data types
    df['bedrooms']=df['bedrooms'].astype('float')
    df.bathrooms=df.bathrooms.replace('10+','10')
    df['bathrooms']=df['bathrooms'].astype('float')
    df.price=df.price.str.replace(',','')
    df['price']=df['price'].astype('float')
    #calculating price_per_unit column
    df.price_per_unit=df.price/df.size
    df.rent_sale=df.rent_sale.str.replace('تجارى','').str.replace('مصايف','').str.replace('و دوبلكس','').str.replace('و أراضى','بيع').str.strip()
    df.rent_sale=df.rent_sale.str.replace('للإيجار','ايجار').str.replace('للبيع','بيع').str.strip()
    df.unit=df.unit.replace({'فيلا منفصلة':'فيلا'})
    df.loc[df.unit == 'محل', 'type'] = "تجاري"
    df.loc[df.unit == 'مبنى تجارى كامل', 'type'] = "تجاري"
    df.loc[df.unit == 'مطعم و كافيه', 'type'] = "تجاري"
    df.loc[df.unit == 'زراعية', 'type'] = "تجاري"
    df.loc[df.unit == 'تجارية', 'type'] = "تجاري"
    df.loc[df.unit == 'جراج', 'type'] = "تجاري"
    df.loc[df.unit == 'مصنع', 'type'] = "تجاري"
    df.loc[df.unit == 'لكل الأغراض', 'type'] = "تجاري"
    df.loc[df.unit == 'مخزن', 'type'] = "تجاري"
    df.loc[df.unit == 'ستوديو', 'type'] = "تجاري"
    df.loc[df.unit == 'عيادة', 'type'] = "تجاري"
    df.loc[df.unit == 'أخرى', 'type'] = "تجاري"
    df.loc[df.unit == 'شقة', 'type'] = "سكني"
    df.loc[df.unit == 'مكتب', 'type'] = "اداري"
    df.loc[df.unit == 'فيلا', 'type'] = "سكني"
    df.loc[df.unit == 'دوبلكس', 'type'] = "سكني"
    df.loc[df.unit == 'شاليه', 'type'] = "سكني"
    df.loc[df.unit == 'سكنية', 'type'] = "سكني"
    df.loc[df.unit == 'غرفة', 'type'] = "سكني"
    df.loc[df.unit == 'تاون هاوس', 'type'] = "سكني"
    df.loc[df.unit == 'بنتهاوس', 'type'] = "سكني"
    df.loc[df.unit == 'توين هاوس', 'type'] = "سكني"
    #droping null values for some columns
    df=df.dropna(subset=['lat'])
    df=df.dropna(subset=['city'])
    df=df[['link','bathrooms','bedrooms','city','location1','location2','price','size','price_per_unit','type','unit','lat','lon','view','description','web_name','source','rent_sale']]
    return df.to_csv(r'olx_cleaning.csv',encoding ='utf-8-sig',index=False)

In[12]:

pensooq call----------------------------

In [ ]:
Opensooq_scraping()
Opensooq_clean()

In[ ]:

In [ ]:
Semsar_scraping()
Semsar_cleaning()

In[ ]:

In [ ]:
tageer_scraping()
tageer_cleaning()

In[ ]:

In [ ]:
olx_scraping()
olx_cleaning()

In[ ]:

In [ ]:
def concat():
    file1=pd.read_csv("Opeensooq_Clean2.csv")
    file2=pd.read_csv("Semsar_clean.csv")
    file3=pd.read_csv("tageer2_cleaning.csv")
    file4=pd.read_csv("olx_cleaning.csv")
    
    df=pd.concat([file1,file2,file3,file4],axis=0)
    df.to_csv("all_data.csv",encoding="utf-8-sig",index=False)
    print(df)

In[ ]:

In [ ]:
concat()

In[ ]: